In [5]:

import trimesh
import os
import open3d as o3d
from PIL import Image
import numpy as np 
import  trimesh
import copy
from utils.utils import *
from utils.trans import *

In [13]:
trans_path = 'scene_2b_colored/mesh_trans'
rotation_matrix = create_rotation_matrix('x',-90)
scene = trimesh.Scene()
pcd_list = []
color_list = []
faces_list = []
def color_the_mesh(mesh,im,uvs):
    # mesh = trimesh.load_mesh('40717.obj')
    # im = Image.open("chair_mesh_40717/red_plastic.png")
    material = trimesh.visual.texture.SimpleMaterial(image=im)
    color_visuals = trimesh.visual.TextureVisuals(uv=uvs, image=im, material=material)
    mesh=trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces, visual=color_visuals, validate=True, process=True)
    return mesh

start_face_idx = 0 
for file_name in os.listdir(trans_path):
    if not 'obj' in file_name:
        continue
    img_path = file_name.split('_')[0] + '_'+ file_name.split('_')[2][:-4] +'.png'
    img = Image.open(os.path.join('meshy_texture',img_path))
    mesh = trimesh.load(os.path.join(trans_path,file_name))
    uvs = mesh.visual.uv
    # Trans late back to Y up-vector system
    
    c_mesh_p.apply_transform(rotation_matrix)
    c_mesh = color_the_mesh(mesh,img,uvs) 
    c_mesh_p = copy.deepcopy(c_mesh)    # copy a mesh for point cloud 
    c_mesh_p.visual = c_mesh_p.visual.to_color()
    verts = np.asarray(c_mesh_p.vertices)
    colors = np.asarray(c_mesh_p.visual.vertex_colors)
    faces = np.asarray(c_mesh_p.faces + start_face_idx)
    colors = colors[:, :3] / 255.0
    pcd_list.append(verts)
    color_list.append(colors)
    scene.add_geometry(c_mesh)
    faces_list.append(faces)
    start_face_idx += verts.shape[0]

## Show point cloud

In [15]:
# vsiaulize the texture secne
scene.show(viewer='gl')

SceneViewer(width=1800, height=1350)

In [10]:
pcds = np.vstack(pcd_list)
pcds_colors = np.vstack(color_list)
pcds_faces = np.vstack(faces_list)
pcd = o3d.geometry.PointCloud()

In [11]:
# Visualize the point cloud

pcd.points = o3d.utility.Vector3dVector(pcds)
pcd.colors = o3d.utility.Vector3dVector(pcds_colors)

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud with Segmentation Labels", width=800, height=600)

In [12]:

# Visualize the mesh

final_mesh = o3d.geometry.TriangleMesh()
final_mesh.vertices = o3d.utility.Vector3dVector(pcds)
final_mesh.vertex_colors = o3d.utility.Vector3dVector(pcds_colors)
final_mesh.triangles = o3d.utility.Vector3iVector(pcds_faces)
o3d.visualization.draw_geometries([final_mesh], window_name="Point Cloud with Segmentation Labels", width=800, height=600)

# o3d.io.write_triangle_mesh("colored_scene/combined_mesh.ply", final_mesh)


## Show Textured Mesh